# Optimizando código en `Python`

Vamos a ilustrar como paralelizar bucles for en Python con un ejemplo:

La idea es paralelizar el siguiente algoritmo KNN para clasificacion basado en la distancia de Gower-BQ (una variante de la distancia de Gower aplicable a conjunto de datos binarios-cuantitativos).

Aqui lo importante no es el funcionamiento del algoritmo en si, si no que hay una parte de este que genera unos costes computacionales altos, lo que se traduce en que al aplicar el algoritmo a un data-set de 5000 variables, este tarde mas de 13 minutos en compilarse por completo. La idea de paralelizarlo es reducir este tiempo de cómputo, para asi hacerlo mas aplicable en contextos reales.

El algoritmo original (sin paralelizar):

In [1]:
import pandas as pd
import numpy as np

In [2]:
def KNN_classification( X , Y , x_new, k, distance = "Gower-BQ" ,  p1=0, p2=0 ):

 
####################################################################################################################################################################################################################################################

    X = pd.concat([X, x_new.to_frame().T], ignore_index=True)

    distances = []

    groups_knn = []

######################################################################################################################################


    if distance == "Gower-BQ":

        def a(Binary_Data) :

            X = Binary_Data

            a = X @ X.T

            return(a)

##########################################################################################

        def d(Binary_Data):

            X = Binary_Data

            ones_matrix = np.ones(( X.shape[0] , X.shape[1])) 

            d = (ones_matrix - X) @ (ones_matrix - X).T

            return(d)

##########################################################################################

        def GowerBQ_Similarity_Python(i,j, BQ_Data_Set, p1, p2):

            X = BQ_Data_Set

   # The data matrix X have to be order in the following way:
   # The p1 first are quantitative, the following p2 are binary categorical 

##########################################################################################
            def G(k, X):

                range = X.iloc[:,k].max() - X.iloc[:,k].min() 

                return(range)

            G_vector = np.repeat(0, p1)

            for r in range(0, p1): 

                G_vector[r] = G(r, X)
##########################################################################################
    
            ones = np.repeat(1, p1)

            Quantitative_Data = X.iloc[: , 0:p1]

            Binary_Data = X.iloc[: , (p1):(p1+p2)]
         
 
##########################################################################################

            numerator_part_1 = ( ones - ( (Quantitative_Data.iloc[i-1,:] - Quantitative_Data.iloc[j-1,:]).abs() / G_vector ) ).sum() 

            numerator_part_2 = a(Binary_Data).iloc[i-1,j-1] 
     
            numerator = numerator_part_1 + numerator_part_2

            denominator = p1 + (p2 - d(Binary_Data).iloc[i-1,j-1])  

            Similarity_Gower = numerator / denominator  

            return(Similarity_Gower)

##########################################################################################


        def Dist_GowerBQ_Py(i, j, BQ_Data ,  p1, p2):

            Dist_Gower = np.sqrt( 1 - GowerBQ_Similarity_Python(i, j, BQ_Data , p1, p2) )

            return(Dist_Gower)

##########################################################################################

        for i in range(1, len(X)):

            distances.append( Dist_GowerBQ_Py( len(X), i , X, p1, p2) )


######################################################################################################################################

######################################################################################################################################
    
    distances = pd.DataFrame({'distances': distances})

    distances = distances.sort_values(by=["distances"]).reset_index(drop=False)
        
    knn = distances.iloc[0:k , :]

    for i in knn.iloc[:,0]:

        groups_knn.append(Y.iloc[i,:])

    unique, counts = np.unique(groups_knn , return_counts=True)

    unique_Y , counts_Y = np.unique(Y , return_counts=True)

    if len(unique) == len(unique_Y) :

        proportions_groups_knn = pd.DataFrame({'proportions_groups': counts/k, 'groups': unique_Y })
    
    elif len(unique) < len(unique_Y) :

        proportions_groups_knn = pd.DataFrame({'proportions_groups': counts/k, 'groups': unique })



    prediction_group = proportions_groups_knn.sort_values(by=["proportions_groups"], ascending=False).iloc[0,:]['groups']

    message = print( "x_new is classify in the group", prediction_group , ". So KNN algorithm predict y_new =",  prediction_group )                                      
                                       

    return proportions_groups_knn , message  

Vamos a probar el algoritmo con un mismo data-set pero con distintos tamaños de filas (observaciones).

Preparamos los datos para la prueba:

In [3]:
Gender_classification = pd.read_csv('gender_classification.csv')

x_new = pd.Series({'long_hair': 1, 'forehead_width_cm': 4, 'forehead_height_cm': 6, 'nose_wide': 1 , 'nose_long': 1 , 'nose_long': 1 , 'lips_thin':1, 'distance_nose_to_lip_long': 1 })

Y = Gender_classification.loc[: , ['gender']]

from sklearn.preprocessing import OrdinalEncoder

ord_enc = OrdinalEncoder()

Y['gender'] = ord_enc.fit_transform(Y[['gender']])

X = Gender_classification.loc[: , ['forehead_width_cm', 'forehead_height_cm',   # Quantitative (2)

                   'long_hair', 'nose_wide', 'nose_long', 'lips_thin', 'distance_nose_to_lip_long'     # Binary (5)
                 
                            ]] 


In [4]:
X_small_1 = X.iloc[0:1000,:]
Y_small_1 = Y.iloc[0:1000,:]

X_small_2 = X.iloc[0:2000,:]
Y_small_2 = Y.iloc[0:2000,:]

X_small_3 = X.iloc[0:3000,:]
Y_small_3 = Y.iloc[0:3000,:]

In [5]:
KNN_classification( X_small_1 , Y_small_1  , x_new, 5 , distance = "Gower-BQ" , p1=2, p2=5 )

KeyboardInterrupt: 

20seg / 27seg / 26.8seg

In [ ]:
KNN_classification( X_small_2 , Y_small_2  , x_new, 5 , distance = "Gower-BQ" , p1=2, p2=5 )

x_new is classify in the group 1.0 . So KNN algorithm predict y_new = 1.0


(   proportions_groups  groups
 0                 1.0     1.0,
 None)

2.47min  / 2.35 / 2.29min

In [ ]:
KNN_classification( X_small_3 , Y_small_3  , x_new, 5 , distance = "Gower-BQ" , p1=2, p2=5 )

x_new is classify in the group 1.0 . So KNN algorithm predict y_new = 1.0


(   proportions_groups  groups
 0                 1.0     1.0,
 None)

8.51min , 8.38min

In [ ]:
KNN_classification( X , Y  , x_new, 5 , distance = "Gower-BQ" , p1=2, p2=5 )

x_new is classify in the group 1.0 . So KNN algorithm predict y_new = 1.0


(   proportions_groups  groups
 0                 1.0     1.0,
 None)

38.48min

In [ ]:
from joblib import Parallel, delayed
import multiprocessing


In [ ]:
def KNN_classification_parallel( X , Y , x_new, k, distance = "Gower-BQ" ,  p1=0, p2=0 ):

 
####################################################################################################################################################################################################################################################

    X = pd.concat([X, x_new.to_frame().T], ignore_index=True)

    distances = []

    groups_knn = []

######################################################################################################################################


    if distance == "Gower-BQ":

        def a(Binary_Data) :

            X = Binary_Data

            a = X @ X.T

            return(a)

##########################################################################################

        def d(Binary_Data):

            X = Binary_Data

            ones_matrix = np.ones(( X.shape[0] , X.shape[1])) 

            d = (ones_matrix - X) @ (ones_matrix - X).T

            return(d)

##########################################################################################

        def GowerBQ_Similarity_Python(i,j, BQ_Data_Set, p1, p2):

            X = BQ_Data_Set

   # The data matrix X have to be order in the following way:
   # The p1 first are quantitative, the following p2 are binary categorical 

##########################################################################################
            def G(k, X):

                range = X.iloc[:,k].max() - X.iloc[:,k].min() 

                return(range)

            G_vector = np.repeat(0, p1)

            for r in range(0, p1): 

                G_vector[r] = G(r, X)
##########################################################################################
    
            ones = np.repeat(1, p1)

            Quantitative_Data = X.iloc[: , 0:p1]

            Binary_Data = X.iloc[: , (p1):(p1+p2)]
         
 
##########################################################################################

            numerator_part_1 = ( ones - ( (Quantitative_Data.iloc[i-1,:] - Quantitative_Data.iloc[j-1,:]).abs() / G_vector ) ).sum() 

            numerator_part_2 = a(Binary_Data).iloc[i-1,j-1] 
     
            numerator = numerator_part_1 + numerator_part_2

            denominator = p1 + (p2 - d(Binary_Data).iloc[i-1,j-1])  

            Similarity_Gower = numerator / denominator  

            return(Similarity_Gower)

##########################################################################################

        def Dist_GowerBQ_Py(i, j, BQ_Data ,  p1, p2):

            Dist_Gower = np.sqrt( 1 - GowerBQ_Similarity_Python(i, j, BQ_Data , p1, p2) )

            return(Dist_Gower)

##########################################################################################

# ESTA ES LA PARTE CRITICA DEL ALGORITMO, LA QUE LO HACE LENTO. ESTE BUCLE FOR ES EL QUE DEBEMOS PARALELIZAR.


        # for i in range(1, len(X)):

           # distances.append( Dist_GowerBQ_Py( len(X), i , X, p1, p2) )

        n_jobs  = multiprocessing.cpu_count()


        distances = Parallel(n_jobs=n_jobs)( delayed(Dist_GowerBQ_Py)( len(X), i , X, p1, p2) for i in range(1, len(X)) )


######################################################################################################################################

######################################################################################################################################
    
    distances = pd.DataFrame({'distances': distances})

    distances = distances.sort_values(by=["distances"]).reset_index(drop=False)
        
    knn = distances.iloc[0:k , :]

    for i in knn.iloc[:,0]:

        groups_knn.append(Y.iloc[i,:])

    unique, counts = np.unique(groups_knn , return_counts=True)

    unique_Y , counts_Y = np.unique(Y , return_counts=True)

    if len(unique) == len(unique_Y) :

        proportions_groups_knn = pd.DataFrame({'proportions_groups': counts/k, 'groups': unique_Y })
    
    elif len(unique) < len(unique_Y) :

        proportions_groups_knn = pd.DataFrame({'proportions_groups': counts/k, 'groups': unique })



    prediction_group = proportions_groups_knn.sort_values(by=["proportions_groups"], ascending=False).iloc[0,:]['groups']

    message = print( "x_new is classify in the group", prediction_group , ". So KNN algorithm predict y_new =",  prediction_group )                                      
                                       

    return proportions_groups_knn , message  

Probamos ahora el algoritmo paralelizado con los mismos ejemplos que antes:

In [ ]:
KNN_classification_parallel( X_small_1 , Y_small_1  , x_new, 5 , distance = "Gower-BQ" , p1=2, p2=5 )

x_new is classify in the group 1.0 . So KNN algorithm predict y_new = 1.0


(   proportions_groups  groups
 0                 1.0     1.0,
 None)

15.5 seg , 16.3seg

In [ ]:
KNN_classification_parallel( X_small_2 , Y_small_2  , x_new, 5 , distance = "Gower-BQ" , p1=2, p2=5 )

x_new is classify in the group 1.0 . So KNN algorithm predict y_new = 1.0


(   proportions_groups  groups
 0                 1.0     1.0,
 None)

1min , 1.03min

In [ ]:
KNN_classification_parallel( X_small_3 , Y_small_3  , x_new, 5 , distance = "Gower-BQ" , p1=2, p2=5 )

x_new is classify in the group 1.0 . So KNN algorithm predict y_new = 1.0


(   proportions_groups  groups
 0                 1.0     1.0,
 None)

3.33min , 3.17min

In [ ]:
KNN_classification_parallel( X , Y  , x_new, 5 , distance = "Gower-BQ" , p1=2, p2=5 )

x_new is classify in the group 1.0 . So KNN algorithm predict y_new = 1.0


(   proportions_groups  groups
 0                 1.0     1.0,
 None)

14.56min

La funcion KNN_classification original tarda mas de 25 mins en hacer validacion cruzada sumple con un set de train de 1000 observaciones. Hay que paralelozarla para hacer viable cross validation con ella !

---

### Codigo sin optimizar:

La justificacion principal de optimizar nuestros codigos en ciencia de datos, poder implementar procedimientos en caso reales que exigen un alto rendimiento computacional.

Si para ejecutar un procedimiento en un caso real el ordenador tarda horas, dias o es incluso intratable computacionalemente, directamente no podremos aplicar dicho procedimiento (por bueno que sea) en casos reales, y tendremos que limitarnos a usar procedimientos quiza peores desde un punto de vista estadistico, pero mejores a nivel de coste computacional.

La idea de optimizar codigo es no tener que desechar buenos procedimientos a nivel estadistico porque sean malos a nivel computacional (sean ineficientes computacionalmente), la idea es hacerlos mas eficientes, para reducir sus costes de computacion.

El siguiente codigo es el que usaremos como ejemplo a lo largo de este articulo, la idea es medir su rendimiento original y posteriormente aplicarle diferentes tecnicas para optimizarlo y ver como cambia su rendimiento.

In [1]:
import pandas as pd
import numpy as np

In [15]:
House_Prices_Data = pd.read_csv('House_Price_Regression.csv')

House_Prices_Data = House_Prices_Data.loc[:, ['price', 'size_in_m_2', 'no_of_bedrooms', 'no_of_bathrooms', 'quality_recode', 'latitude', 'private_garden_recode', 'private_pool_recode', 'longitude']]

House_Prices_Data['quality_recode'] = House_Prices_Data['quality_recode'].astype('object')
House_Prices_Data['private_garden_recode'] = House_Prices_Data['private_garden_recode'].astype('object')
House_Prices_Data['private_pool_recode'] = House_Prices_Data['private_pool_recode'].astype('object')

House_Prices_Data = House_Prices_Data.rename({'price': 'Y'}, axis=1)

Data_Houses_Prices_Train = House_Prices_Data.sample(frac=0.8, replace=False, weights=None, random_state=555, axis=None, ignore_index=True)

Data_Houses_Prices_Test = House_Prices_Data.drop( Data_Houses_Prices_Train.index , )

###########################################################################

# Ahora vamos a sacar por un lado los X (predictores) de test y train , y por otro Y (respuesta) de test y train

# Ademas vamos a generar un nuevo Data_Test=[Y_test, X_tes]  y Data_Train=[Y_train, X_train]

# Con estos data frames son con los que nos vamos a manejar

X_test = Data_Houses_Prices_Test.loc[: , ['size_in_m_2', 'no_of_bedrooms', 'no_of_bathrooms','latitude', 'longitude', 'private_garden_recode', 'private_pool_recode',  'quality_recode']]

Y_test = Data_Houses_Prices_Test.loc[: , 'Y']

Data_Test = pd.concat([Y_test , X_test], axis=1)

#########################

X_train = Data_Houses_Prices_Train.loc[: , ['size_in_m_2', 'no_of_bedrooms', 'no_of_bathrooms','latitude', 'longitude', 'private_garden_recode', 'private_pool_recode',  'quality_recode']]

Y_train = Data_Houses_Prices_Train.loc[: , 'Y']

Data_Train = pd.concat([Y_train , X_train], axis=1)

In [3]:
# Como ejemplo de x_new (nueva observacion de los predictores) cogemos la sexta (5 en python) observacion de X_test

x_new = X_test.iloc[ 5 , range(0, X_test.shape[1])]

In [4]:
X_train_small = X_train.iloc[0:100]
Y_train_small = Y_train.iloc[0:100]

La siguiente funcion es una implementacion de KNN para regresion usando la distancia de Gower, que es una metrica computacionalmente muy costosa.

La idea es usar la funcion KNN_regresion dentro de una rutina de validacion cruzada, como ejemplo de procedimiento que supone un alto coste computacional.

In [5]:
def KNN_regression( X  , Y  , x_new, k, distance, p1, p2, p3 ):

   
####################################################################################################################################################################################################################################################

    # Y tiene que ser una variable respuesta cuantitativa

    # X tiene que ser un panda data frame con los predictotres (X1,...,Xp). 

    # x_new tiene que ser un vector. 

####################################################################################################################################################################################################################################################

    X  = pd.concat([X , x_new.to_frame().T], ignore_index=True)

    distances = []

    Y_values_knn = []


####################################################################################################################################################################################################################################################
    
    
    
    if distance == "Gower":

        # The data matrix X have to be order in the following way:
        # The p1 first are quantitative, the following p2 are binary categorical, and the following p3 are multiple categorical.


        def a(Binary_Data) :

            X = Binary_Data

            a = X @ X.T

            return(a)

##########################################################################################

        def d(Binary_Data):

            X = Binary_Data

            ones_matrix = np.ones(( X.shape[0] , X.shape[1])) 

            d = (ones_matrix - X) @ (ones_matrix - X).T

            return(d)

##########################################################################################

        def alpha_py(i,j, Multiple_Categorical_Data):

                X = Multiple_Categorical_Data

                alpha = np.repeat(0, X.shape[1])

                for k in range(0, X.shape[1]) :

                    if X.iloc[i-1, k] == X.iloc[j-1, k] :

                        alpha[k] = 1

                    else :

                        alpha[k] = 0


                alpha = alpha.sum()

                return(alpha)

   ##########################################################################################


        def Gower_Similarity_Python(i,j, Mixed_Data_Set, p1, p2, p3):

            X = Mixed_Data_Set

       # The data matrix X have to be order in the following way:
       # The p1 first are quantitative, the following p2 are binary categorical, and the following p3 are multiple categorical.

   #####################################################################################
        
            def G(k, X):

                range = X.iloc[:,k].max() - X.iloc[:,k].min() 

                return(range)

            G_vector = np.repeat(0.5, p1)

            for r in range(0, p1):

                G_vector[r] = G(r, X)
                
      
    ##########################################################################################
    
            ones = np.repeat(1, p1)

            Quantitative_Data = X.iloc[: , 0:p1]

            Binary_Data = X.iloc[: , (p1):(p1+p2)]
            
            Multiple_Categorical_Data = X.iloc[: , (p1+p2):(p1+p2+p3) ]

    ##########################################################################################

            numerator_part_1 = ( ones - ( (Quantitative_Data.iloc[i-1,:] - Quantitative_Data.iloc[j-1,:]).abs() / G_vector ) ).sum() 

            numerator_part_2 = a(Binary_Data).iloc[i-1,j-1] + alpha_py(i,j, Multiple_Categorical_Data)

            numerator = numerator_part_1 + numerator_part_2
 
            denominator = p1 + (p2 - d(Binary_Data).iloc[i-1,j-1]) + p3

            Similarity_Gower = numerator / denominator  

            return(Similarity_Gower)

##########################################################################################

        def Dist_Gower_Py(i, j, Mixed_Data , p1, p2, p3):

            Dist_Gower = np.sqrt( 1 - Gower_Similarity_Python(i, j, Mixed_Data , p1, p2, p3) )

            return(Dist_Gower)    

    ###################################################################

        for j in range(1, len(X)):

            distances.append( Dist_Gower_Py( len(X), j , X, p1, p2, p3) )

        
######################################################################################################################################
    
    distances = pd.DataFrame({'distances': distances})

    distances = distances.sort_values(by=["distances"]).reset_index(drop=False)
        
    knn = distances.iloc[0:k , :]

    for i in knn.iloc[:,0]:

        Y_values_knn.append(Y.iloc[i , ])


    y_predict = sum(Y_values_knn)/k


                                     
    return y_predict   , distances

In [36]:
y_predict  , distances = KNN_regression( X_train= X_train_small , Y_train= Y_train_small , x_new=x_new, k=10,  distance="Gower" , p1=5, p2=2, p3=1 )

NameError: name 'X' is not defined

In [32]:
y_predict

1526288.8

In [33]:
distances

,index,distances
0,21,0.258714
1,28,0.415606
2,89,0.419265
3,95,0.424388
4,38,0.425896
...,...,...
95,4,0.687463
96,86,0.698281
97,58,0.733435
98,94,0.754767


La siguiente rutina de validacion simple usando la funcion KNN_regression es la que usaremos como ejemplo a lo largo de este artuiculo, ahora podremos ver cuanto tarda en ejecutarse con el algoritmo original sin haber sido optimizado de ningun modo, despues repetiremos el proceso habiendo optimizado el algoritmo con diferentes procedimientos:

In [10]:
def validacion_simple(Data_Test, X_train, Y_train):

    ##########################

    def prediction(i, Data_Test, X_train, Y_train ):

     x_new = Data_Test.iloc[ i , range(1, Data_Test.shape[1])]
 
     y_new_predict = KNN_regression( X_train  , Y_train , x_new, k=10, distance = "Gower" , p1=5, p2=2, p3=1  )

     return(y_new_predict)

    ##########################

    y_predictions_vector = []

    for i in  range(0, len(Data_Test)):

        y_new_predict = prediction(i, Data_Test, X_train, Y_train )

        y_predictions_vector.append( y_new_predict )

    ##########################

    ECM = ( (y_predictions_vector - Data_Test.loc[: , 'Y'])**2 ).sum() 

 
    return(y_predictions_vector , ECM)

In [11]:
y_predictions_vector, ECM = validacion_simple(Data_Houses_Prices_Test, X_train_small , Y_train_small)

KeyboardInterrupt: 

In [10]:
ECM

3054953133867935.5

El tiempo de computacion usando solo las 100 primeras filas de X_train e Y_train es de  5.30 / 5.14 mins minutos

Si usasemos X e Y enteras (que tienen  1524 filas) el tiempo de computacion seria demasiado grande como para usar el algoritmo en la practica. 

---

### Paralelizacion de bucles for

Usaremos la libreria `joblib` para paralelizar los bucles for del algoritmo KNN_regression, tambien haremos lo mismo con el bloque de codigo con el que hacemos la validacion simple

In [7]:
def KNN_regression_Parallel_Pandas( X , Y , x_new, k, distance  , p1=0, p2=0, p3=0 ):

##########################################################################################################

## Para paralelizar el algoritmo 

    from joblib import Parallel, delayed
    import multiprocessing

####################################################################################################################################################################################################################################################

    # Y tiene que ser una variable respuesta cuantitativa

    # X tiene que ser un panda data frame con los predictotres (X1,...,Xp). 

    # x_new tiene que ser un vector. 

####################################################################################################################################################################################################################################################

    X  = pd.concat([X , x_new.to_frame().T], ignore_index=True)
 
    distances = []
    Y_values_knn = []

####################################################################################################################################################################################################################################################
    
    
    
    if distance == "Gower":

        # The data matrix X have to be order in the following way:
        # The p1 first are quantitative, the following p2 are binary categorical, and the following p3 are multiple categorical.


        def a(Binary_Data) :

            X = Binary_Data

            a = X @ X.T

            return(a)

##########################################################################################

        def d(Binary_Data):

            X = Binary_Data

            ones_matrix = np.ones(( X.shape[0] , X.shape[1])) 

            d = (ones_matrix - X) @ (ones_matrix - X).T

            return(d)

##########################################################################################

        def alpha_py(i,j, Multiple_Categorical_Data):

                X = Multiple_Categorical_Data

                alpha = np.repeat(0, X.shape[1])

                for k in range(0, X.shape[1]) :

                    if X.iloc[i-1, k] == X.iloc[j-1, k] :

                        alpha[k] = 1

                    else :

                        alpha[k] = 0


                alpha = alpha.sum()

                return(alpha)

   ##########################################################################################


        def Gower_Similarity_Python(i,j, Mixed_Data_Set, p1, p2, p3):

            X = Mixed_Data_Set

   # The data matrix X have to be order in the following way:
   # The p1 first are quantitative, the following p2 are binary categorical, and the following p3 are multiple categorical.

   #####################################################################################
        
            def G(k, X):

                range = X.iloc[:,k].max() - X.iloc[:,k].min() 

                return(range)

            G_vector = np.repeat(0.5, p1)

            for r in range(0, p1):

                G_vector[r] = G(r, X)
                
      
    ##########################################################################################
    
            ones = np.repeat(1, p1)

            Quantitative_Data = X.iloc[: , 0:p1]

            Binary_Data = X.iloc[: , (p1):(p1+p2)]
            
            Multiple_Categorical_Data = X.iloc[: , (p1+p2):(p1+p2+p3) ]

    ##########################################################################################

            numerator_part_1 = ( ones - ( (Quantitative_Data.iloc[i-1,:] - Quantitative_Data.iloc[j-1,:]).abs() / G_vector ) ).sum() 

            numerator_part_2 = a(Binary_Data).iloc[i-1,j-1] + alpha_py(i,j, Multiple_Categorical_Data)

            numerator = numerator_part_1 + numerator_part_2
 
            denominator = p1 + (p2 - d(Binary_Data).iloc[i-1,j-1]) + p3

            Similarity_Gower = numerator / denominator  

            return(Similarity_Gower)

##########################################################################################

        def Dist_Gower_Py(i, j, Mixed_Data , p1, p2, p3):

            Dist_Gower = np.sqrt( 1 - Gower_Similarity_Python(i, j, Mixed_Data , p1, p2, p3) )

            return(Dist_Gower)    

    ###################################################################

    ## PARTE DEL CODIGO A PARALELIZAR

        # for i in range(1, len(X)):

            # distances.append( Dist_Gower_Py( len(X), i , X, p1, p2, p3) )


        n_jobs  = multiprocessing.cpu_count()

        distances = Parallel(n_jobs=n_jobs)( delayed(Dist_Gower_Py)( len(X), s , X, p1, p2, p3) for s in range(1, len(X)) )

######################################################################################################################################
    

    distances = pd.DataFrame({'distances': distances})

    distances = distances.sort_values(by=["distances"]).reset_index(drop=False)
        
    knn = distances.iloc[0:k , :]

    for i in knn.iloc[:,0]:

        Y_values_knn.append(Y.iloc[i , ])


    y_predict = sum(Y_values_knn)/k


                                     
    return y_predict  , distances 

In [8]:
y_predict  , distances = KNN_regression_Parallel_Pandas( X=X_train_small , Y=Y_train_small , x_new=x_new, k=10,  distance="Gower" , p1=5, p2=2, p3=1 )

In [9]:
y_predict

1526288.8

In [10]:
distances

,index,distances
0,21,0.258714
1,28,0.415606
2,89,0.419265
3,95,0.424388
4,38,0.425896
...,...,...
95,4,0.687463
96,86,0.698281
97,58,0.733435
98,94,0.754767


In [11]:
def validacion_simple_Parallel_Pandas(Data_Test, X_train, Y_train):

    ##########################

    from joblib import Parallel, delayed
    import multiprocessing

    n_jobs  = multiprocessing.cpu_count()

    ##########################

    def prediction(i, Data_Test, X_train, Y_train ):

     x_new = Data_Test.iloc[ i , range(1,Data_Test.shape[1])]

     # Usamos KNN_regression_Parallel en vez de KNN_regression
 
     y_new_predict , distances = KNN_regression_Parallel_Pandas( X=X_train  , Y=Y_train , x_new=x_new, k=10, distance = "Gower" , p1=5, p2=2, p3=1  )
     

     return(y_new_predict)

    ##########################

    

    # Paralelizamos el siguiente bucle for :

    # for i in  range(0, len(Data_Test)):

        # y_new_predict = prediction(i, Data_Test, X_train, Y_train )

        # y_predictions_vector.append( y_new_predict )

    y_predictions_vector = []

    y_predictions_vector = Parallel(n_jobs=n_jobs)( delayed(prediction)( i, Data_Test, X_train, Y_train) for i in range(0, len(Data_Test)) )

    #########################

    ECM = ( (y_predictions_vector - Data_Test.loc[: , 'Y'])**2 ).sum() 


    return(y_predictions_vector, ECM)

In [18]:
y_predictions_vector , ECM = validacion_simple_Parallel_Pandas(Data_Test=Data_Test, X_train=X_train_small, Y_train=Y_train_small)

In [19]:
ECM

3054953133867935.5

En este caso, tras paralelizar los algoritmos el tiempo de computacion se reduce de 5.30 mins a 1.42 / 1.34 mins , lo cual es bastante.

---

### Usar `Numpy` en vez de `Pandas`

In [20]:
def KNN_regression_Parallel_Numpy( X , Y , x_new, k, distance, p1=0, p2=0, p3=0 ):

##########################################################################################################

## Para paralelizar el algoritmo 

    from joblib import Parallel, delayed
    import multiprocessing

    n_jobs  = multiprocessing.cpu_count()

####################################################################################################################################################################################################################################################

    # Y, X y x_new deben ser objetos Pandas ya que luego seran convertidos a objetos Numpy automaticamente por el algoritmo
    
    # Y tiene que ser un Pandas data frame con la variable respuesta 

    # X tiene que ser un Pandas data frame con los predictotres (X1,...,Xp). 

    # x_new tiene que ser un vector con una nueva observacion de los predictores. 

####################################################################################################################################################################################################################################################

    Y = Y.to_numpy()

    X = X.to_numpy() 

    x_new = x_new.to_numpy()

    X = np.concatenate((X, [x_new]), axis=0)


    distances = []

    Y_values_knn = []


####################################################################################################################################################################################################################################################
    
    
    
    if distance == "Gower":

        # The data matrix X have to be order in the following way:
        # The p1 first are quantitative, the following p2 are binary categorical, and the following p3 are multiple categorical.


        def a(Binary_Data) :

            X = Binary_Data

            a = X @ X.T

            return(a)

##########################################################################################

        def d(Binary_Data):

            X = Binary_Data

            ones_matrix = np.ones(( X.shape[0] , X.shape[1])) 

            d = (ones_matrix - X) @ (ones_matrix - X).T

            return(d)

##########################################################################################

        def alpha_py(i,j, Multiple_Categorical_Data):

            X = Multiple_Categorical_Data

            alpha = np.repeat(0, X.shape[1])

            def argumento_bucle_for(k):

                if X[i-1, k] == X[j-1, k] :

                    alpha[k] = 1

                else :

                    alpha[k] = 0

                return(alpha) 
    
            alpha=Parallel(n_jobs=n_jobs)( delayed(argumento_bucle_for)( k ) for k in range(0, X.shape[1]) )
    
            alpha = sum(alpha)

            return(alpha)

##########################################################################################

        def Gower_Similarity_Python(i,j, Mixed_Data_Set, p1, p2, p3):

            X = Mixed_Data_Set

   # The data matrix X have to be order in the following way:
   # The p1 first are quantitative, the following p2 are binary categorical, and the following p3 are multiple categorical.

   #####################################################################################
        
            def G(k, X):

                range = X[:,k].max() - X[:,k].min() 

                return(range)

            G_vector = np.repeat(0.5, p1)

            for r in range(0, p1):

                G_vector[r] = G(r, X)
                
      
    ##########################################################################################
    
            ones = np.repeat(1, p1)

            Quantitative_Data = X[: , 0:p1]

            Binary_Data = X[: , (p1):(p1+p2)]
            
            Multiple_Categorical_Data = X[: , (p1+p2):(p1+p2+p3) ]

    ##########################################################################################

            numerator_part_1 = ( ones - ( abs(Quantitative_Data[i-1,:] - Quantitative_Data[j-1,:]) / G_vector ) ).sum() 

            numerator_part_2 = a(Binary_Data)[i-1,j-1] + alpha_py(i,j, Multiple_Categorical_Data)

            numerator = numerator_part_1 + numerator_part_2
 
            denominator = p1 + (p2 - d(Binary_Data)[i-1,j-1]) + p3

            Similarity_Gower = numerator / denominator  

            return(Similarity_Gower)
  
##########################################################################################

        def Dist_Gower_Py(i, j, Mixed_Data , p1, p2, p3):

            Dist_Gower = np.sqrt( 1 - Gower_Similarity_Python(i, j, Mixed_Data , p1, p2, p3) )

            return(Dist_Gower)    

    ###################################################################

    ## PARTE DEL CODIGO A PARALELIZAR

        #for j in range(1, len(X)):

          #distances.append( Dist_Gower_Py( len(X), j , X, p1, p2, p3) )

        n_jobs  = multiprocessing.cpu_count()

        distances = Parallel(n_jobs=n_jobs)( delayed(Dist_Gower_Py)( len(X), s , X, p1, p2, p3) for s in range(1, len(X)) )

######################################################################################################################################
    
    distances = pd.DataFrame({'distances': distances})

    distances = distances.sort_values(by=["distances"]).reset_index(drop=False)
        
    knn = distances.iloc[0:k , :]

    for i in knn.iloc[:,0]:

        Y_values_knn.append(Y[i, ])


    y_predict = sum(Y_values_knn)/k

                                     
    return y_predict  , distances 

In [21]:
y_predict  , distances = KNN_regression_Parallel_Numpy( X=X_train_small , Y=Y_train_small , x_new=x_new, k=10, distance="Gower", p1=5, p2=2, p3=1)

In [22]:
y_predict

1526288.8

In [23]:
distances

,index,distances
0,21,[0.2587138029633777]
1,28,[0.41560562680697666]
2,89,[0.4192652667104365]
3,95,[0.4243881064105595]
4,38,[0.42589561616225]
...,...,...
95,4,[0.6874630024733788]
96,86,[0.6982814288066614]
97,58,[0.7334352997561012]
98,94,[0.7547667737679794]


In [29]:
def validacion_simple_Parallel_Numpy(Data_Test, X_train, Y_train):

    ##########################

    from joblib import Parallel, delayed
    import multiprocessing

    n_jobs  = multiprocessing.cpu_count()

    ##########################

    def prediction(i, Data_Test, X_train, Y_train ):

     x_new = Data_Test.iloc[ i , range(1 , Data_Test.shape[1])]

     # Usamos KNN_regression_Parallel en vez de KNN_regression
 
     y_new_predict , distances = KNN_regression_Parallel_Numpy( X_train  , Y_train , x_new, k=10, distance = "Gower" , p1=5, p2=2, p3=1  )

     return(y_new_predict)

    ##########################

    y_predictions_vector = []

    # Paralelizamos el siguiente bucle for :

    # for i in  range(0, len(Data_Test)):

        # y_new_predict = prediction(i, Data_Test, X_train, Y_train )

        # y_predictions_vector.append( y_new_predict )

    
    y_predictions_vector = Parallel(n_jobs=n_jobs)( delayed(prediction)( i, Data_Test, X_train, Y_train) for i in range(0, len(Data_Test)) )

    #########################

    ECM = ( (y_predictions_vector - Data_Test.loc[: , 'Y'])**2 ).sum() 


    return(y_predictions_vector, ECM)

In [30]:
y_predictions_vector, ECM = validacion_simple_Parallel_Numpy(Data_Test=Data_Test, X_train=X_train_small , Y_train=Y_train_small)

In [31]:
ECM

3054953133867935.5

La opcion paralelizada y con Numpy solo 26 seg , mientras que la parelizada pero con Pandas 1.40 min, y la no paralelizada y con pandas 5.30 min

### Optimizando codigo con `Numba`

### Alternativas a los bucles for